In [2]:
import sys
sys.path.insert(0, '/home/jovyan/Proj/ml_combiner/TSEBRA/bin')
import os
import csv
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from datetime import datetime
import matplotlib.pyplot as plt
from genome_anno import Anno
from overlap_graph import Graph
import itertools as it
from evidence import Evidence
from gnn import GNN
import pickle
import tensorflow as tf
import subprocess as sp
import random
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [3]:
species = ['Caenorhabditis_elegans','Parasteatoda_tepidariorum', 
                 'Arabidopsis_thaliana', 
                 'Bombus_terrestris', 'Rhodnius_prolixus',
                'Populus_trichocarpa', 'Danio_rerio', 
               'Drosophila_melanogaster', 'Medicago_truncatula', 'Tetraodon_nigroviridis']
species.sort()
parent_dir = '/home/jovyan/Proj/ml_combiner/data/train_data'
out = '/home/jovyan/brain/test_data/ml_training/'

def get_cds_keys(tx):
    keys = [tx.chr, tx.strand] + [str(c[0]) + '_' + str(c[1]) \
        for c in tx.get_type_coords('CDS', frame=False)]
    return keys

In [4]:
graph_list = []
for s in species:
    with open(f'{out}/train_data/{s}_graph_object.pkl', 'rb') as inp:
    #with open(f'{out}/train_data/{s}_altseqs_graph_object.pkl', 'rb') as inp:
        g = pickle.load(inp)
        graph_list.append(g)

In [ ]:
g = graph_list[0]
s = species[0]

for s, g in zip(species, graph_list):
    ref_anno = Anno(f'{out}/{s}/annot.gtf', 'reference')
    ref_anno.addGtf()
    ref_anno_keys = []
    ref_anno_cds = []
    for tx in ref_anno.transcripts.values():
        cds_keys = get_cds_keys(tx)
        ref_anno_cds += cds_keys[2:]
        ref_anno_keys.append('_'.join(cds_keys))
    ref_anno_keys = set(ref_anno_keys)
    ref_anno_cds = set(ref_anno_cds)

    tp = 0
    for c in g.connected_components().values():
        all_keys = []
        cds_set = []
        chr = ''
        strand = ''
        for n in c:
            tx = g.__tx_from_key__(n)    
            chr = tx.chr
            strand = tx.strand
            cds_set += tx.get_type_coords('CDS', frame=False)    
        cds_set = list(set([f'{i[0]}_{i[1]}' for i in cds_set]))
        cds_set = [i for i in cds_set if i in ref_anno_cds]
        cds_set.sort()
        for i in range(1, len(cds_set)+1):
            for subset in it.combinations(cds_set, i):
                #all_keys.append(f'{chr}_{strand}_' + '_'.join(list(subset)))
                if f'{chr}_{strand}_' + '_'.join(list(subset)) in ref_anno_keys:
                    tp += 1
        #tp += len(set(all_keys).intersection(ref_anno_keys))
        #print(tp)
    print(s, tp, len(ref_anno), tp/len(ref_anno))
    with open(f'{out}/tmp.test', 'a+') as file:
        file.write(f'{s}, {tp}, {len(ref_anno)}, {tp/len(ref_anno)}')

In [14]:
list(ref_anno_cds)[0]

'7238880_7239473'